# DSCI 525: Milestone 1
In this notebook, we download rainfall data from [figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681) in a zip file, extract all the csv files and combine them into a single file. The comparison of time taken to combine these files across different laptops is provided. EDA is done on the combined file and techniques used to reduce memory usage are implemented in python. In the end, the dataframe is transferred from python to R using `enter-approach-name`.

## 1. Importing packages

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import pyarrow as pa
import rpy2_arrow.pyarrow_rarrow as pyra

## 2. Download data with the figshare API 

In [2]:
current_path = os.getcwd()
current_path

'C:\\Users\\rowan\\DSCI_525\\group_14\\notebooks'

Here, we specify the article we want to import from `figshare` with the website url and the output directory.

In [3]:
article_id = 14096681  
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

We download the file `data.zip` to our local computer.

In [5]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 21.1 s
Wall time: 2min 20s


Now, we unzip all the csv files from the zipped file.

In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 14 s
Wall time: 14.1 s


In [7]:
%%time
### just listing to get an idea how individual file looks like 
df_1 = pd.read_csv("figsharerainfall/ACCESS-CM2_daily_rainfall_NSW.csv")
df_1

CPU times: total: 1.06 s
Wall time: 1.09 s


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


Removing the file named `observed_daily_rainfall_SYD.csv` in the data folder.

In [8]:
os.remove(output_directory + 'observed_daily_rainfall_SYD.csv')

## 3. Combining CSV files

Changing the directory to enter the folder with the csv files.

In [9]:
os.chdir(current_path + '/figsharerainfall/')

In [10]:
%%time
import pandas as pd
use_cols = df_1.columns
files = glob.glob('*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r'^[^_]+(?=_)', file)[0])
                for file in files)
              )
df.to_csv("combined_data.csv")

CPU times: total: 5min 53s
Wall time: 5min 53s


In [11]:
%%sh
du -sh combined_data.csv

5.7G	combined_data.csv


### <center >Comparison of time taken to load and combine the files
    
| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Mel         |  macOS Big Sur   |16 GB| M1, 8-core|   Yes  |  5min 55s  |
| Rohit       |  macOS Monterey  | 8 GB| M1, 8-core|   Yes  |  5min 22s  |
| Rowan       |Windows 10 Education | 16 GB    |i7-10750H  |  Yes   |    5min 53s        | 

## 4. EDA on the combined data set

In [12]:
df.shape

(62467843, 6)

In [13]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


### Changing dtypes

In [14]:
%%time

types = {'lat_min': 'float16', 
          'lat_max': 'float16', 
          'lon_min': 'float16', 
          'lon_max': 'float16',
          'rain (mm/day)':'float16',
          'file':'str'}

df = pd.read_csv("combined_data.csv",index_col=0, parse_dates=True, dtype=types)

print(df.info(memory_usage='deep'))
print(df["model"].value_counts())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float16
 1   lat_max        float16
 2   lon_min        float16
 3   lon_max        float16
 4   rain (mm/day)  float16
 5   model          object 
dtypes: float16(5), object(1)
memory usage: 5.0 GB
None
MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR 

### <center >Comparison of time taken to change dtype
    
| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Mel         |  macOS Big Sur   |16 GB| M1, 8-core|   Yes  |  1min 1s     |
| Rohit       |  macOS Monterey  | 8 GB| M1, 8-core|   Yes  |  2min 25s  |
| Rowan       |Windows 10 Education | 16 GB    |i7-10750H  |  Yes   | 1min 28s           | 

### Selecting Columns

In [16]:
%%time
use_cols = ['lat_min','lat_max','lon_min','lon_max','rain (mm/day)','model']
df = pd.read_csv("combined_data.csv",usecols=use_cols)
print(df['model'].value_counts())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: total: 45.5 s
Wall time: 46.9 s


### <center >Comparison of time taken when selecting columns
    
| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Mel         |  macOS Big Sur   |16 GB| M1, 8-core|   Yes  |  42s     |
| Rohit       |  macOS Monterey  | 8 GB| M1, 8-core|   Yes  | 2min 1s   |
| Rowan       |Windows 10 Education | 16 GB    |i7-10750H  |  Yes   | 46s       | 

### Loading Chunks

In [26]:
%%time

result = None
for chunk in pd.read_csv("combined_data.csv", chunksize=1000):
    chunk_model = chunk[
        "model"]
    chunk_result = chunk_model.value_counts()
    if result is None:
        result = chunk_result
    else:
        result = result.add(chunk_result, fill_value=0)

result.sort_values(ascending=False, inplace=True)

print(f"{result} \n")

MPI-ESM1-2-HR       5154240.0
TaiESM1             3541230.0
NorESM2-MM          3541230.0
CMCC-CM2-HR4        3541230.0
CMCC-CM2-SR5        3541230.0
CMCC-ESM2           3541230.0
SAM0-UNICON         3541153.0
FGOALS-f3-L         3219300.0
GFDL-CM4            3219300.0
GFDL-ESM4           3219300.0
EC-Earth3-Veg-LR    3037320.0
MRI-ESM2-0          3037320.0
BCC-CSM2-MR         3035340.0
MIROC6              2070900.0
ACCESS-CM2          1932840.0
ACCESS-ESM1-5       1610700.0
INM-CM5-0           1609650.0
INM-CM4-8           1609650.0
KIOST-ESM           1287720.0
FGOALS-g3           1287720.0
MPI-ESM1-2-LR        966420.0
NESM3                966420.0
AWI-ESM-1-1-LR       966420.0
MPI-ESM-1-2-HAM      966420.0
NorESM2-LM           919800.0
BCC-ESM1             551880.0
CanESM5              551880.0
Name: model, dtype: float64 

CPU times: total: 5min 13s
Wall time: 5min 13s


### <center >Comparison of time taken when loading chunks
    
| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Mel         |  macOS Big Sur   |16 GB| M1, 8-core|   Yes  |  2min 21s     |
| Rohit       |  macOS Monterey  | 8 GB| M1, 8-core|   Yes  |  5min 14s  |
| Rowan       |Windows 10 Education | 16 GB    |i7-10750H  |  Yes   | 5min 13s       | 

## EDA in R
[Discuss why you chose this approach over others]  
We have decided to go with the parquet file approach to transfer the dataframe from python to R.  
As discussed in the lecture, using a CSV and plain zip file reduces our ability to perform EDA on the file as it is 'zipped' whilst parquet files are able to compress data to a similar storage size whilst providing us with the freedom to read the data quickly. Furthermore, due to its columnar storage, it reduces the I/O and network traffic to greatly speed up reading and querying times.

In [17]:
df = pd.read_csv('combined_data.csv', index_col=0, parse_dates=True)

In [18]:
%%time
df.to_parquet('combined_data.parquet')

CPU times: total: 16.6 s
Wall time: 18.3 s


In [19]:
%%time
%load_ext rpy2.ipython

df = pd.read_parquet('combined_data.parquet')

C:\Users\rowan\miniconda3\envs\525_2022\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


CPU times: total: 10.2 s
Wall time: 9.69 s


In [25]:
%%time
%%R
library(dplyr)
options(repr.matrix.max.cols=150, repr.matrix.max.rows=200)

df <- arrow::open_dataset("combined_data.parquet", 
                          format="parquet", 
                          partitioning=c("model"))
df |>
    group_by(model) |>
    summarize(count=n()) |>
    ungroup() |>
    collect()

R[write to console]: 
Attaching package: 'dplyr'


R[write to console]: The following objects are masked from 'package:stats':

    filter, lag


R[write to console]: The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




# A tibble: 27 x 2
   model              count
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# ... with 17 more rows
CPU times: total: 3.36 s
Wall time: 3.41 s


### <center >Comparison of transferring dataframe from python to R
    
| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Mel         |  macOS Big Sur   |16 GB| M1, 8-core|   Yes  |  3.88s     |
| Rohit       |  macOS Monterey  | 8 GB| M1, 8-core|   Yes  | 8min 3s   |
| Rowan       |Windows 10 Education | 16 GB    |i7-10750H  |  Yes   |     3.41s       | 